In [40]:
from pathlib import Path
from scipy import sparse
from scipy.sparse import csr_matrix
import pandas as pd

from implicit.datasets.lastfm import get_lastfm

In [81]:
def load_data(file: Path) -> csr_matrix:
    df = pd.read_csv(file, sep=",")

    df.set_index(['userID', 'restaurantID'], inplace=True)
    coo = sparse.coo_matrix(
        (
            df["rating"].astype(float),
            (df.index.get_level_values(0), df.index.get_level_values(1)),
        )
    )
	
    return coo.tocsr()

In [82]:
csr = load_data('user-data.csv')

print(csr)

  (1, 662)	4.0
  (1, 671)	1.0
  (1, 910)	1.0
  (1, 1473)	4.0
  (1, 1527)	2.0
  (1, 2068)	1.0
  (1, 2370)	5.0
  (1, 2906)	5.0
  (1, 2908)	2.0
  (1, 3344)	8.0
  (1, 3419)	1.0
  (1, 3499)	3.0
  (1, 3674)	1.0
  (1, 3742)	5.0
  (1, 4129)	9.0
  (1, 4223)	3.0
  (1, 4469)	7.0
  (1, 4772)	1.0
  (1, 4840)	8.0
  (1, 5527)	1.0
  (1, 5905)	2.0
  (1, 6166)	4.0
  (1, 6209)	8.0
  (1, 6242)	9.0
  (1, 6393)	1.0
  :	:
  (4999, 57405)	2.0
  (4999, 57805)	1.0
  (4999, 58006)	3.0
  (4999, 58097)	8.0
  (4999, 58330)	4.0
  (4999, 58445)	4.0
  (4999, 58467)	8.0
  (4999, 59031)	3.0
  (4999, 59123)	5.0
  (4999, 59341)	2.0
  (4999, 59352)	4.0
  (4999, 59763)	3.0
  (4999, 60108)	3.0
  (4999, 60286)	6.0
  (4999, 60421)	7.0
  (4999, 60634)	8.0
  (4999, 61526)	4.0
  (4999, 61603)	5.0
  (4999, 62075)	6.0
  (4999, 62122)	1.0
  (4999, 62755)	4.0
  (4999, 62901)	4.0
  (4999, 62948)	5.0
  (4999, 62984)	8.0
  (4999, 63059)	5.0
